In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import glob
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
import os
os.getcwd()

'/home/jupyter'

In [3]:
os.chdir("./Data")

In [4]:
#uploading files
aisles = pd.read_csv("aisles.csv", dtype={'aisle_id': np.uint8, 'aisle': 'category'})
departments = pd.read_csv("departments.csv", dtype={'department_id': np.uint8, 'department': 'category'})
orders = pd.read_csv("orders.csv", dtype = {'order_id': np.uint32,
                                            'user_id': np.uint32,
                                            'eval_set': 'category',
                                            'order_number': np.uint8,
                                            'order_dow': np.uint8,
                                            'order_hour_of_day': np.uint8})
orders_products_prior = pd.read_csv("order_products__prior.csv", dtype={'order_id': np.uint32,
                                                                        'product_id': np.uint16,
                                                                        'add_to_cart_order': np.uint8,
                                                                        'reordered': bool})
orders_products_train = pd.read_csv("order_products__train.csv", dtype={'order_id': np.uint32,
                                                                        'product_id': np.uint16,
                                                                        'add_to_cart_order': np.uint8,
                                                                        'reordered': bool})
products = pd.read_csv("products.csv", dtype={'product_id': np.uint16,
                                              'aisle_id': np.uint8,
                                              'department_id': np.uint8})

In [5]:
data_list = [aisles, departments, orders, 
             orders_products_prior, orders_products_train, products]
data_list_name = ['aisles', 'departments', 'orders', 
'orders_products_prior', 'orders_products_train', 'products']

j=0
for i in data_list:
    i.name = data_list_name[j]
    j += 1

In [6]:
#Checking for null values
#Creating a function to find null values
def null_columns(x):
    y = x.columns[x.isnull().any()]
    return y

In [7]:
for dataset in data_list:
    if len(null_columns(dataset)) == 0:
        print('Dataset ' + dataset.name + ' has no null values ')
    else:
        print('Dataset '+ dataset.name + ' has null values in column '+ str([i for i in null_columns(dataset)]))

Dataset aisles has no null values 
Dataset departments has no null values 
Dataset orders has null values in column ['days_since_prior_order']
Dataset orders_products_prior has no null values 
Dataset orders_products_train has no null values 
Dataset products has no null values 


In [8]:
#Dataset orders has null values in column ['days_since_prior_order'] 
# Other Columns have no null values
#Finding the number of null values in orders.days_since_prior_order
sum(orders.days_since_prior_order.isnull())

206209

In [9]:
#There are 206209 null values
#Lets examine the null values
orders.loc[orders.days_since_prior_order.isnull()==True,['user_id', 'order_number']]

,user_id,order_number
0,1,1
11,2,1
26,3,1
39,4,1
45,5,1
50,6,1
54,7,1
75,8,1
79,9,1
83,10,1


In [10]:
#days_since_prior_order attribue for first order for all the users is null so it can be changed to 0

orders.days_since_prior_order = orders.days_since_prior_order.fillna(int(0))

## Feature Engineering

In [11]:
prior_prods = pd.DataFrame()
prior_prods['prior_orders'] = orders_products_prior.groupby(orders_products_prior.product_id).size().astype(np.int32)
prior_prods['prior_reorders'] = orders_products_prior['reordered'].groupby(orders_products_prior.product_id).sum().astype(np.float32)
prior_prods['prior_reorder_rate'] = (prior_prods.prior_reorders / prior_prods.prior_orders).astype(np.float32)

In [12]:
prior_products = products.join(prior_prods, on='product_id')

In [13]:
prior_products['is_gluten_free'] = prior_products['product_name'].str.contains('gluten' and 'free') * 1
prior_products['is_organic'] = prior_products['product_name'].str.contains('organic' or 'Organic') * 1
prior_products['is_low_fat'] = prior_products['product_name'].str.contains('low' and 'fat') * 1

In [14]:
prior_avg_add_to_cart = orders_products_prior.groupby('product_id')['add_to_cart_order'].mean().to_frame('prior_avg_add_to_cart').astype(np.float32).reset_index()

In [15]:
prior_products = prior_products.merge(prior_avg_add_to_cart, how='left', left_on='product_id', right_on='product_id')

In [16]:
prior_products = prior_products.replace([np.inf, -np.inf], np.nan)

In [17]:
prior_products = prior_products.fillna(0)
# prior_products.to_pickle('prior_products')

In [18]:
#Merging with products
orders_products_prior = orders_products_prior.merge(products, on='product_id')
orders_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,True,Organic Egg Whites,86,16
1,26,33120,5,False,Organic Egg Whites,86,16
2,120,33120,13,False,Organic Egg Whites,86,16
3,327,33120,5,True,Organic Egg Whites,86,16
4,390,33120,28,True,Organic Egg Whites,86,16


In [19]:
#Merging with products
#Adding Order info to prior orders
orders_products_prior = orders_products_prior.merge(orders, on='order_id')
orders_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,True,Organic Egg Whites,86,16,202279,prior,3,5,9,8.0
1,2,28985,2,True,Michigan Organic Kale,83,4,202279,prior,3,5,9,8.0
2,2,9327,3,False,Garlic Powder,104,13,202279,prior,3,5,9,8.0
3,2,45918,4,True,Coconut Butter,19,13,202279,prior,3,5,9,8.0
4,2,30035,5,False,Natural Sweetener,17,13,202279,prior,3,5,9,8.0


### User_features

In [20]:
#User Features
user = pd.DataFrame()
user['mean_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
user['number_of_orders'] = orders.groupby('user_id').size().astype(np.int16)
user = user.reset_index()

In [21]:
#User features based on prior orders
prior_users = pd.DataFrame()
prior_users['prior_total_items'] = orders_products_prior.groupby('user_id').size().astype(np.int16)
prior_users['prior_all_products'] = orders_products_prior.groupby('user_id')['product_id'].apply(set)
prior_users['total_distinct_products'] = prior_users.prior_all_products.map(len).astype(np.int16)

prior_users = prior_users.reset_index()
prior_users = prior_users.merge(user, on = 'user_id')
del user
prior_users['average_products_per_basket'] = (prior_users.prior_total_items / prior_users.number_of_orders).astype(np.float32)
prior_users = pd.DataFrame(prior_users)
prior_users.head()

,user_id,prior_total_items,prior_all_products,total_distinct_products,mean_days_between_orders,number_of_orders,average_products_per_basket
0,1,59,"{17122, 196, 46149, 26405, 14084, 13032, 26088...",18,17.272728,11,5.363636
1,2,195,"{45066, 2573, 18961, 1559, 32792, 23, 22559, 1...",102,15.200000,15,13.000000
2,3,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33,11.076923,13,6.769231
3,4,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17,14.166667,6,3.000000
4,5,37,"{11777, 28289, 40706, 48775, 20754, 6808, 1398...",23,9.200000,5,7.400000


In [22]:
# prior_users.to_pickle('./Created Datasets/prior_users')

### Product Features

In [23]:
#Top_100_reordered_products_prior
top_100_reordered_products = pd.DataFrame(orders_products_prior.loc[orders_products_prior.reordered == 
                                                       True].groupby('product_id').size().sort_values(ascending=False).head(100).reset_index(name='counts'))
top_100_reordered_products_list = list(top_100_reordered_products.product_id)

#Top_30_reordered_aisle_prior
top_30_reordered_aisles = pd.DataFrame(orders_products_prior.loc[orders_products_prior.reordered ==
                                                                 True].groupby('aisle_id').size().sort_values(ascending=False).head(30).reset_index(name='counts'))
top_30_reordered_aisles_list = list(top_30_reordered_aisles.aisle_id)

#Top_10_reordered_deparments_prior
top_10_reordered_department = pd.DataFrame(orders_products_prior.loc[orders_products_prior.reordered == 
                                                       True].groupby('department_id').size().sort_values(ascending=False).head(10).reset_index(name='counts'))
top_10_reordered_departments_list = list(top_10_reordered_department.department_id)

In [24]:
# top_100_reordered_products.to_pickle('./Created Datasets/top_100_reordered_products')
# top_30_reordered_aisles.to_pickle('./Created Datasets/top_30_reordered_aisles')
# top_10_reordered_department.to_pickle('./Created Datasets/top_10_reordered_department')

In [25]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,0.0
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


### User X Product Features

In [26]:
#User X Product features
orders_products_prior['user_product'] = orders_products_prior.product_id + orders_products_prior.user_id * 100000


In [27]:
userXproduct = orders_products_prior.groupby('user_product').size().astype(np.uint16).reset_index(name = 'nb_counts')
a = orders_products_prior.groupby('user_product')['order_id'].max().astype(np.uint16).reset_index(name = 'last_order_id')
avg_pos_in_cart = orders_products_prior.groupby('user_product')['add_to_cart_order'].mean().astype(np.float16).reset_index(name = 'mean_pos_in_cart')
userXproduct.user_product = userXproduct.user_product.astype(np.uint32)

In [28]:
userXproduct = userXproduct.merge(a, on='user_product')

In [29]:
userXproduct = userXproduct.merge(avg_pos_in_cart, on='user_product')
userXproduct.head()

,user_product,nb_counts,last_order_id,mean_pos_in_cart
0,462,3,1173,7.332031
1,1077,2,26608,21.000000
2,2857,2,18199,13.500000
3,3176,3,10708,14.000000
4,3210,1,24744,8.000000


In [30]:
# userXproduct.to_pickle('./Created Datasets/userXproduct')

In [31]:
#Train test Split
test_orders = orders[orders.eval_set == 'test']
train_orders = orders[orders.eval_set == 'train']

In [32]:
def create_dummies(df, colname):
    col_dummies = pd.get_dummies(df[colname], prefix=colname)
    col_dummies.drop(col_dummies.columns[0], axis=1, inplace=True)
    df = pd.concat([df, col_dummies], axis=1)
    df.drop( colname, axis = 1, inplace = True )
    return df

In [33]:
orders_products_train_1 = orders_products_train.copy()
orders_products_train_1.set_index(['order_id', 'product_id'], inplace=True)

In [34]:
def features(selected_orders, labels_given=False):
    print('build candidate list')
    order_list = []
    product_list = []
    labels = []
    i=0
    for row in selected_orders.itertuples():
        i+=1
        if i%10000 == 0: print('order row',i)
        order_id = row.order_id
        user_id = row.user_id
        user_products = prior_users.loc[prior_users.user_id == user_id, 'prior_all_products'].values[0]
        product_list += user_products
        order_list += [order_id] * len(user_products)
        if labels_given:
            labels += [(order_id, product) in orders_products_train_1.index for product in user_products]
    df = pd.DataFrame({'order_id':order_list, 'product_id':product_list}, dtype=np.int32)
    if labels_given:
         df['y'] = labels            
    del order_list
    del product_list
    
    print('user related features')
    df['user_id'] = df.order_id.map(orders.user_id).astype(np.uint32)
    df = df.merge(prior_users, on = 'user_id').drop('prior_all_products', axis=1)
    
    
    print('order related features')
    orders_x = orders[['order_id', 'order_dow','order_hour_of_day', 'days_since_prior_order']]
    df = df.merge(orders_x, on = 'order_id')
    df['days_since_ratio'] = (df.days_since_prior_order / df.mean_days_between_orders).astype(np.float32)
    del orders_x
    
    print('product related features')
    prior_products_x = prior_products[['product_id', 'aisle_id', 'department_id',
       'prior_orders', 'prior_reorders', 'prior_reorder_rate',
       'is_gluten_free', 'is_organic', 'is_low_fat', 'prior_avg_add_to_cart']]
    df = df.merge(prior_products_x, on='product_id')
    del prior_products_x

   
    print('user_X_product related features')
    df['user_product'] = df.product_id + df.user_id * 100000
    #df = df.merge(userXproduct, on='user_product')
    df['UP_orders'] = df.user_product.map(userXproduct.nb_counts)
    df['UP_last_order_id'] = df.user_product.map(userXproduct.last_order_id)
    df['UP_mean_pos_in_cart'] = df.user_product.map(userXproduct.mean_pos_in_cart)
    #df = df.rename(columns = {'nb_counts' : 'UP_orders', 'last_order_id': 'UP_last_order_id', 'mean_pos_in_cart': 'UP_mean_pos_in_cart'})
    df['UP_orders_ratio'] = (df.UP_orders / df.number_of_orders).astype(np.float16)
    df['UP_reorder_rate'] = (df.UP_orders / df.number_of_orders).astype(np.float16)
    df['UP_orders_since_last'] = df.number_of_orders - df.UP_last_order_id.map(orders.order_number)
    df.drop(['UP_last_order_id', 'user_product', 'user_id'], axis=1, inplace=True)
    
    print('Dummy Variables')
    df = create_dummies(df, 'order_dow')
    df = create_dummies(df, 'order_hour_of_day')
    
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna(-1)
    
    return (df, labels)

def top_features(df):
    targ_pre_product = 'is_product_'
    targ_pre_department = 'is_department_'
    targ_pre_aisle = 'is_aisle_'

    print('Top 100 Products')
    for product in top_100_reordered_products_list:
        df[targ_pre_product + str(product)] = df.product_id == product
    print('Top 10 Departments')
    for department in top_10_reordered_departments_list:
        df[targ_pre_department+ str(department)] = df.department_id == department
    print('Top 30 Aisles')
    for aisle in top_30_reordered_aisles_list:
        df[targ_pre_aisle+ str(aisle)] = df.aisle_id == aisle
    is_columns = df.columns[df.columns.str.contains('is_')]
    df[is_columns] = df[is_columns].astype(np.uint8)
    

    del targ_pre_product
    del targ_pre_department
    del targ_pre_aisle
    del is_columns
    return df

    

In [36]:
df_train, labels = features(train_orders, labels_given=True)

build candidate list
order row 10000
order row 20000
order row 30000
order row 40000
order row 50000
order row 60000
order row 70000
order row 80000
order row 90000
order row 100000
order row 110000
order row 120000
order row 130000
user related features
order related features
product related features
user_X_product related features
Dummy Variables


In [36]:
# df_train = top_features(df_train)

Top 100 Products
Top 10 Departments
Top 30 Aisles


In [37]:
for column, data in df_train.items():
    if np.issubdtype(df_train[column], np.int64):
        df_train[column] = df_train[column].astype(np.uint32)
    elif np.issubdtype(df_train[column], np.float64):
        df_train[column] = df_train[column].astype(np.float32)

In [38]:
df_train.to_pickle('./Created Datasets/df_train')

In [44]:
for column, data in df_test.items():
    if np.issubdtype(df_test[column], np.int64):
        df_test[column] = df_test[column].astype(np.uint32)
    elif np.issubdtype(df_test[column], np.float64):
        df_test[column] = df_test[column].astype(np.float32)

In [45]:
df_test.to_pickle('./Created Datasets/df_test')

In [41]:
df_train.dtypes

order_id                         int64
product_id                       int64
y                                 bool
prior_total_items                int64
total_distinct_products          int64
mean_days_between_orders       float32
number_of_orders                 int64
average_products_per_basket    float32
days_since_prior_order         float64
days_since_ratio               float32
aisle_id                         int64
department_id                    int64
prior_orders                   float64
prior_reorders                 float32
prior_reorder_rate             float32
is_gluten_free                   uint8
is_organic                       uint8
is_low_fat                       uint8
prior_avg_add_to_cart          float32
UP_orders                      float64
UP_mean_pos_in_cart            float16
UP_orders_ratio                float16
UP_reorder_rate                float16
UP_orders_since_last           float64
order_dow_1                      int64
order_dow_2              